# Preparation: Combine Dataset

- We've alrealy got several csv files, in which 500 pages are stored in each file. 
- Now, we'll combine all these csv files to 'combined_data.csv'.

In [ ]:
import os
import glob
import pandas as pd

#find all the csv files under the same folder and put them in the all_csv_list
all_csv='csv'
all_csv_list=[i for i in glob.glob('*.{}'.format(all_csv))]   

#combine the csv files in all_csv_list
combined_csv = pd.concat([pd.read_csv(i) for i in all_csv_list])     
combined_csv.to_csv("combined_data.csv", index=False)

# Data Cleaning

In [99]:
df = pd.read_csv('combined_data.csv')

## (1) Delete empty data & repeated data

In [100]:
df = df[~df["id"].isin(["none"])]
df = df.drop_duplicates(subset=['id'], keep='first')
df['id'].is_unique

True

## (2)Add unique index: 'case'

In [101]:
df['case'] = range(len(df))
df.index = df['case']
df.columns

Index(['Unnamed: 0', 'id', 'uniqid', 'schoolname', 'professional', 'countries',
       'sid', 'pid', 'nid', 'offer_id', 'uid', 'degree', 'degree_other',
       'apply_results', 'apply_lang', 'year', 'timestamp', 'updatetime',
       'semester', 'notice_time', 'vieworder', 'state', 'urls', 'toefl', 'gre',
       'ielts', 'ptea', 'sub', 'gmat', 'undergraduate_subject',
       'undergraduate_sid', 'undergraduate_gpa', 'graduate_subject', 'case'],
      dtype='object')

## (3) Timestamp conversion:
convert 'notice_time' to：
  - 'time_year'
  - 'time_month
  - 'time_day'

In [102]:
df.to_csv('clean1.csv')

In [103]:
df = pd.read_csv('clean1.csv')
# make sure: type(df['notice_time'][i]) - int64, otherwise - str

/Users/yangpeiwen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [104]:
import time

time_year = []
time_month = []
time_day = []

#year
for i in range(len(df)):
    timeArray= time.localtime(df['notice_time'][i])
    #otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    otherStyleTime = time.strftime("%Y", timeArray)
    time_year.append(otherStyleTime)
    #print(type(otherStyleTime)) -> str

#month
for i in range(len(df)):
    timeArray= time.localtime(df['notice_time'][i])
    otherStyleTime = time.strftime("%m", timeArray)
    time_month.append(otherStyleTime)
    
#day
for i in range(len(df)):
    timeArray= time.localtime(df['notice_time'][i])
    otherStyleTime = time.strftime("%d", timeArray)
    time_day.append(otherStyleTime)
    
df.insert(len(df.columns), 'time_year', time_year)  
df.insert(len(df.columns), 'time_month', time_month)  
df.insert(len(df.columns), 'time_day', time_day) 

In [105]:
df.columns

Index(['case', 'Unnamed: 0', 'id', 'uniqid', 'schoolname', 'professional',
       'countries', 'sid', 'pid', 'nid', 'offer_id', 'uid', 'degree',
       'degree_other', 'apply_results', 'apply_lang', 'year', 'timestamp',
       'updatetime', 'semester', 'notice_time', 'vieworder', 'state', 'urls',
       'toefl', 'gre', 'ielts', 'ptea', 'sub', 'gmat', 'undergraduate_subject',
       'undergraduate_sid', 'undergraduate_gpa', 'graduate_subject', 'case.1',
       'time_year', 'time_month', 'time_day'],
      dtype='object')

## (4) Delete useless Columns & Rows

In [106]:
df.to_csv('clean2.csv')

In [107]:
df = pd.read_csv('clean2.csv')
# make sure: type(df['time_year'][i]) - int64, otherwise - str

/Users/yangpeiwen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [108]:
# delete useless columns
df = df.drop(columns=['uniqid','pid','nid','offer_id','degree_other','apply_lang','timestamp','updatetime','vieworder','state','undergraduate_subject','toefl','gre','ptea','sub','gmat','undergraduate_gpa','graduate_subject'])

# delete useless raws
df = df.drop(df[(df.time_year < 2015)].index)

In [109]:
df.columns

Index(['Unnamed: 0', 'case', 'Unnamed: 0.1', 'id', 'schoolname',
       'professional', 'countries', 'sid', 'uid', 'degree', 'apply_results',
       'year', 'semester', 'notice_time', 'urls', 'ielts', 'undergraduate_sid',
       'case.1', 'time_year', 'time_month', 'time_day'],
      dtype='object')

## (5) Split 'ielts' -> get 'ieltsz'
'ieltsz': total grade of IELTS

In [110]:
df['ielts'].head(5)

0    {'r': '7', 'l': '7', 's': '6', 'w': '7', 'z': ...
1    {'r': '8.0', 'l': '7.5', 's': '5.5', 'w': '6.0...
2        {'r': '', 'l': '', 's': '', 'w': '', 'z': ''}
3        {'r': '', 'l': '', 's': '', 'w': '', 'z': ''}
4        {'r': '', 'l': '', 's': '', 'w': '', 'z': ''}
Name: ielts, dtype: object

In [111]:
ielts_r = []
ielts_l = []
ielts_s = []
ielts_w = []
ielts_z = []

for i in range(len(df)):     
    a = []
    br = []
    bl = []
    bs = []
    bw = []
    bz = []
    cr = []
    cl = []
    cs = []
    cw = []
    cz = []
    
    try:
        item = df['ielts'][i]
        #print(item)
        a = item.split(",")

        br = a[0].split(":")
        bl = a[1].split(":")
        bs = a[2].split(":")
        bw = a[3].split(":")
        bz = a[4].split(":")

        cr = br[1].strip(" ' ")
        cl = bl[1].strip(" ' ")
        cs = bs[1].strip(" ' ")
        cw = bw[1].strip(" ' ")
        cz = bz[1].strip(" ' ").strip(" } ").strip(" ' ")
    
        ielts_r.append(cr)
        ielts_l.append(cl)
        ielts_s.append(cs)
        ielts_w.append(cw)
        ielts_z.append(cz)
    
    except:
        cr = 'none'
        cl = 'none'
        cs = 'none'
        cw = 'none'
        cz = 'none'
    
        ielts_r.append(cr)
        ielts_l.append(cl)
        ielts_s.append(cs)
        ielts_w.append(cw)
        ielts_z.append(cz)

In [112]:
data_ielts = {
    'IELTS_R': ielts_r,
    'IELTS_L': ielts_l,
    'IELTS_S': ielts_s,
    'IELTS_W': ielts_w,
    'IELTS_Z': ielts_z
}

pd_data_ielts = pd.DataFrame.from_dict(data_ielts)
pd_data_ielts.head(5)

,IELTS_R,IELTS_L,IELTS_S,IELTS_W,IELTS_Z
0,7,7,6,7,7
1,8.0,7.5,5.5,6.0,7
2,,,,,
3,,,,,
4,,,,,


In [ ]:
pd_data_ielts.to_csv('data_for_ielts.csv')

In [ ]:
# Manual cleaning with data_for_ielts.csv

In [113]:
dfielts = pd.read_csv('data_for_ielts.csv')

In [114]:
# insert 'ieltsz' into df
df.insert(len(df.columns), 'ieltsz', dfielts['IELTS_Z']) 

In [115]:
df.columns

Index(['Unnamed: 0', 'case', 'Unnamed: 0.1', 'id', 'schoolname',
       'professional', 'countries', 'sid', 'uid', 'degree', 'apply_results',
       'year', 'semester', 'notice_time', 'urls', 'ielts', 'undergraduate_sid',
       'case.1', 'time_year', 'time_month', 'time_day', 'ieltsz'],
      dtype='object')

## (6) Tagging: 'apply_results'
- 0 - reject
- 1 - offer
- 2 - waiting

In [116]:
#Original tags: 1-offer; 2-AD小奖; 3-AD无奖; 4-被拒; 5-waiting list
df.apply_results.value_counts()

1    36146
3    28221
4    14535
2     7813
5     2056
Name: apply_results, dtype: int64

In [117]:
df['apply_results'].replace(4, 0, inplace=True)

In [118]:
df['apply_results'].replace([1, 2, 3], 1, inplace=True)

In [119]:
df['apply_results'].replace(5, 2, inplace=True)

In [120]:
df.apply_results.value_counts()

1    72180
0    14535
2     2056
Name: apply_results, dtype: int64

In [121]:
df.to_csv('clean3.csv')

## (7) Tagging: 'undergraduate_sid'
- 1 - 985
- 2 - 211
- 3 - neither 985 or 211

### ① Crawl the List of 985 Universities

In [24]:
#Crawl the List of 985 Universities
from lxml import etree
from selenium import webdriver
import time

sch985 = []

bro = webdriver.Chrome(executable_path='/Users/yangpeiwen/Desktop/semesterA/Day3/groupwork/chromedriver')
#bro = webdriver.Chrome(executable_path='YOUR PATH')
url = 'https://daxue.eol.cn/985.shtml'
bro.get(url)
time.sleep(2)
page_text = bro.page_source

tree = etree.HTML(page_text)
div_list = tree.xpath('/html/body/div[3]/div[2]/div[2]/table/tbody/tr')
for div in div_list:
    try:
        sch = div.xpath('./td[2]/a/text()')[0]
    except:
        sch = div.xpath('./td[1]/a/text()')[0]
    sch985.append(sch)

### ② Crawl the List of 211 Universities

In [25]:
#Crawl the List of 211 Universities
from lxml import etree
from selenium import webdriver
import time

sch211 = []

bro = webdriver.Chrome(executable_path='/Users/yangpeiwen/Desktop/semesterA/Day3/groupwork/chromedriver')
#bro = webdriver.Chrome(executable_path='YOUR PATH')
url = 'https://daxue.eol.cn/211.shtml'
bro.get(url)
time.sleep(2)
page_text = bro.page_source

tree = etree.HTML(page_text)
div_list = tree.xpath('/html/body/div[3]/div[2]/div[3]/table/tbody/tr')
for div in div_list:
    try:
        sch = div.xpath('./td[2]/a/text()')[0]
    except:
        sch = div.xpath('./td[1]/a/text()')[0]
    sch211.append(sch)

### ③ Supplementary the Lists

In [26]:
#neither 985 or 211
schneither = ['双非','长沙理工大学','华东政法大学','西南政法大学','南方科技大学','北京建筑大学','双非一本','南方科技大学','北京建筑大学','深圳大学','华南农业大学','北京电影学院','上海对外经贸大学','外交学院','西安建筑科技大学','北京语言大学','中国科技大学','南京林业大学','青岛大学','西北政法大学','南京信息工程大学','中国科学院大学/ 丹麦哥本哈根大学','一本双非',' 山东财经大学',' 华东政法大学','中央美术学院','中科院','广东工业大学','国内中外合办大学','华政','科学院','南方医科大学','南京工业大学','沈阳体育学院','四川外国语大学','四川美术学院','浙江工业大学','中国科学院大学','中科院某研究所','北京第二外国语学院','北京师范大学珠海分校','东北财经大学','中科院地理所','一本双非','依然双非','浙江传媒','浙江传媒学院','西政','西南政法','西北政法大学','天津体育学院','四川大学锦城学院','广东外语外贸大学','河南科技大学','华侨大学','空军工程大学','南京信息工程大学','中国戏曲学院','中国民航大学','西南林业大学','扬州大学','双非二本','首都师范大学','山西大学','沈阳建筑大学','山东科技大学','青岛大学','青岛理工大学',' 深圳大学','安徽财经大学','安徽农业大学','非985，非211，','二流大学','福建工程学院','湖北工业大学','河北科技师范学院','南京邮电大学','普通二本','天津大学仁爱学院','四川外国语大学（非211，985）','西安外国语大学','宁波诺丁汉大学','华北计算技术研究所','国际关系学院','北京工商大学嘉华学院']

#985
add985 = ['985&211','211/985','G5','哈尔滨工业大学（威海）','哈工大','国内985','中流985','北京师范大学-香港浸会大学联合国际学院',' 中国人民大学',' 北京航空航天大学','HKUST','北京大学','985', ' 东北大学',' 浙江大学','北航','Top 20 985','中上985','211，985',' 复旦大学','985211','985，211',' 清华大学',' 四川大学','211&985','985 & 211','985/211','wuhan university','Xiamen Univerisity','北大/清华/人大/复旦','某前十985','清/北/人/复/交','清北复交','浙江大学 211 & 985','浙大/电子科大/哈工大',' 上海交通大学','C9','SYSU','复旦','中山大学， The Hong Kong Polytechnic University','中興大學（台灣）／同濟大學','浙大/复旦/南大/南开/武大/华中科大/上海交大/天大','西北工业大学 985',' 吉林大学',' 华中科技大学','985+211','985， 双一流','RUC','复旦/浙大/南大/交大/南开/武大/川大','哈工大威海','南京大学约翰·霍普金斯大学中美文化研究中心','清华',' 哈尔滨工业大学（威海）',' 厦门大学','985 211','THU','WHU','北清科武复交','末流985','人大','普通理工科985','211985']
sch985.extend(add985)

#211
add211 = ['211','普通211',' 华南农业大学','212',' 苏州大学',' 中国政法大学','双非一本  省属211','一线城市211','中国政法大学/211',' 暨南大学',' 天津医科大学','211高校','211院校','211法学院','南京师范大学211','211大学','北邮','北邮+伦敦玛丽女王大学','北京邮电大学+QMUL','中部某女子师范211','中南财经政法大学+滑铁卢大学（2+2）']
sch211.extend(add211)

### ④ Tagging

In [122]:
df = pd.read_csv('clean3.csv')

In [123]:
#1 - 985; 2 - 211; 3 - neither 985 or 211

for i in range(len(df)):
    if df['undergraduate_sid'][i] in sch985:
        df['undergraduate_sid'][i] = '1'
        #df['undergraduate_sid'][i].replace(df['undergraduate_sid'][i],'1')
    if df['undergraduate_sid'][i] in sch211:
        df['undergraduate_sid'][i] = '2'
    if df['undergraduate_sid'][i] in schneither:
        df['undergraduate_sid'][i] = '3'

<ipython-input-123-cc5510a5a4e6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['undergraduate_sid'][i] = '3'
<ipython-input-123-cc5510a5a4e6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['undergraduate_sid'][i] = '2'
<ipython-input-123-cc5510a5a4e6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['undergraduate_sid'][i] = '1'


In [124]:
df.undergraduate_sid.value_counts().head(60)

2                                             18175
3                                              6975
1                                              2377
香港中文大学                                           76
The Chinese University of Hong Kong              60
美研                                               40
City University of Hong Kong                     39
美本                                               38
香港大学                                             38
University of Toronto                            37
University College London                        37
CUHK                                             37
University of Auckland                           36
UWM                                              32
HKU                                              31
Columbia University                              30
美国                                               28
National University of Singapore                 27
UW-Madison                                       26
美硕          

In [125]:
df.to_csv('data_clean.csv')

# Please be noticed that:

### we used this 'data_clean.csv' file above as the data source for most of our data visualization. 
### However, some of the visualization was drawn using another file ('index_new_time.csv') as the data source, and the process of how to get that file will be shown later.

### Please rest assured that there's no essential difference between these two files. The only difference are the number of reserved columns, the order of the colums and the number of reserved rows.

### In other words, compared to 'data_clean.csv', the 'index_new_time.csv' can be regarded as a not fully processed file, and the missing cleaning process will be supplemented in the visualization process with the same processing criteria if we use this file rather than 'data_clean.csv'.

 ### Here shows the process of how to get 'index_new_time.csv'：

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('combined_data.csv')

In [15]:
df = df[~df["id"].isin(["none"])]
df = df.drop_duplicates(subset=['id'], keep='first')
df['id'].is_unique

True

In [17]:
df['case'] = range(len(df))
df.drop(columns=['uniqid','pid','nid','degree_other','vieworder','state','timestamp','updatetime']) 
df.head(3)

,Unnamed: 0,id,uniqid,schoolname,professional,countries,sid,pid,nid,offer_id,...,gre,ielts,ptea,sub,gmat,undergraduate_subject,undergraduate_sid,undergraduate_gpa,graduate_subject,case
0,0,120433,aWGCambCyvPv,The Chinese University of Hong Kong,Master of Comparative and Public History,香港,308,0,951,43964,...,"{'v': '', 'q': '', 'aw': '', 'z': ''}","{'r': '7', 'l': '7', 's': '6', 'w': '7', 'z': ...","{'r': '', 'l': '', 's': '', 'w': '', 'z': ''}","{'zy': '0', 'z': ''}","{'q': '', 'v': '', 'z': ''}",广播电视工程,双非,GPA 3.87/4.00,NaN,0
1,1,120432,Kbey9ajufbeq,Imperial College London,Artificial Intelligence and Machine Learning,英国,328,0,720,43963,...,"{'v': '', 'q': '', 'aw': '', 'z': ''}","{'r': '8.0', 'l': '7.5', 's': '5.5', 'w': '6.0...","{'r': '', 'l': '', 's': '', 'w': '', 'z': ''}","{'zy': '0', 'z': ''}","{'q': '', 'v': '', 'z': ''}",软件工程,211,87、17%,NaN,1
2,2,120431,0eemrrWSf8Gj,KU Leuven,Mechanical Engineering,比利时,670,495,1050,43962,...,"{'v': '163', 'q': '170', 'aw': '4.0', 'z': '333'}","{'r': '', 'l': '', 's': '', 'w': '', 'z': ''}","{'r': '', 'l': '', 's': '', 'w': '', 'z': ''}","{'zy': '0', 'z': ''}","{'q': '', 'v': '', 'z': ''}",New Energy Science and Engineering,211,87/100,Energy Science and Technology,2


In [5]:
df.to_csv('df111_retest.csv')

In [6]:
df = pd.read_csv('df111_retest.csv')

/Users/yangpeiwen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df.index = df['case']
df.to_csv('dfindex_retest.csv')

In [8]:
df = pd.read_csv('dfindex_retest.csv')

/Users/yangpeiwen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
type(df['notice_time'][1])

numpy.int64

In [10]:
import time

time_year = []
time_month = []
time_day = []

#year
for i in range(len(df)):
    timeArray= time.localtime(df['notice_time'][i])
    #otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    otherStyleTime = time.strftime("%Y", timeArray)
    time_year.append(otherStyleTime)
    #print(type(otherStyleTime)) -> str

#month
for i in range(len(df)):
    timeArray= time.localtime(df['notice_time'][i])
    otherStyleTime = time.strftime("%m", timeArray)
    time_month.append(otherStyleTime)
    
#day
for i in range(len(df)):
    timeArray= time.localtime(df['notice_time'][i])
    otherStyleTime = time.strftime("%d", timeArray)
    time_day.append(otherStyleTime)

In [11]:
df.insert(20, 'time_year', time_year)  
df.insert(21, 'time_month', time_month)  
df.insert(22, 'time_day', time_day) 
df.columns

Index(['case', 'Unnamed: 0', 'Unnamed: 0.1', 'id', 'uniqid', 'schoolname',
       'professional', 'countries', 'sid', 'pid', 'nid', 'offer_id', 'uid',
       'degree', 'degree_other', 'apply_results', 'apply_lang', 'year',
       'timestamp', 'updatetime', 'time_year', 'time_month', 'time_day',
       'semester', 'notice_time', 'vieworder', 'state', 'urls', 'toefl', 'gre',
       'ielts', 'ptea', 'sub', 'gmat', 'undergraduate_subject',
       'undergraduate_sid', 'undergraduate_gpa', 'graduate_subject', 'case.1'],
      dtype='object')

In [12]:
df['time_year']

0        2020
1        2021
2        2021
3        2021
4        2021
         ... 
96071    2018
96072    2018
96073    2018
96074    2018
96075    2018
Name: time_year, Length: 96076, dtype: object

In [ ]:
import pandas as pd
df.to_csv('index_new_time.csv')